In [ ]:
import pandas as pd
import numpy as np
import os
import json
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Data/gtddata.csv')

In [ ]:
for i in df.columns:
  if df[i].dtype == 'object':
    if len(df[i].unique()) > 10000:
      print(i)

city
summary
corp1
target1
scite1


In [ ]:
df[['corp1', 'target1','scite1']].iloc[0]

corp1                                Cairo Police Department
target1                            Cairo Police Headquarters
scite1     "Police Chief Quits," Washington Post, January...
Name: 0, dtype: object

In [ ]:
x = df.target1.value_counts()
type(x)

pandas.core.series.Series

In [ ]:
for i in df.target1.value_counts():
  print(i)

Streaming output truncated to the last 5000 lines.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

In [ ]:
corpus = df.target1
for c in corpus:
    urls = re.findall('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', c)
    if len(urls) == 0:
        pass
    else:
        print(f'Description: {list(corpus).index(c)} - Links: {urls}')

In [ ]:
# Replacing sites and hiperlinks
corpus_wo_hiperlinks = []
for c in corpus:
    c = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', 'link', c)
    corpus_wo_hiperlinks.append(c)

# Replacing numbers
corpus_wo_numbers = []
for c in corpus_wo_hiperlinks:
    c = re.sub('\d+(?:\.\d*(?:[eE]\d+))?', 'number', c)
    corpus_wo_numbers.append(c)
corpus_wo_numbers[399]

# Replacing special characteres with whitespace
corpus_text = []
for c in corpus_wo_numbers:
    c = re.sub(r'\W', ' ', c)
    corpus_text.append(c)
corpus_text[1113]

cleaned_corpus = pd.Series(corpus_text).apply(lambda x: x.lower())
cleaned_corpus[990]

cleaned_corpus.index = df['target1'].index

df['target1'] = cleaned_corpus

df['target1']

0                                cairo police headquarters
1                                          edes substation
2        r o t c  offices at university of wisconsin  m...
3        selective service headquarters in madison wisc...
4                 army recruiting station  denver colorado
                               ...                        
93515                                            positions
93516                                            personnel
93517                                     electrical pylon
93518                    reverend  father elijah juma wada
93519                                               casino
Name: target1, Length: 93520, dtype: object

In [ ]:
!pip install transformers

import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.8 MB/s eta 0:00:00


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased").to(device)

In [ ]:
dfs = []
for i in range(0, len(df), 100):
  j = i+100
  if j>len(df)-1:
    j = len(df)
  dfs.append(df.iloc[i:j])

In [ ]:
tokenizeds = []
c = 1
max_length = 0

# Assuming dfs is a list of dataframes or similar structure
for i in dfs:
    tokenized = tokenizer(i.target1.to_list(), padding=True, truncation=True, return_tensors="pt")

    # Update the maximum sequence length if needed
    max_length = max(max_length, tokenized['input_ids'].shape[1])

    tokenizeds.append(tokenized)
    print(c, '/', len(dfs))
    c += 1

1 / 936
2 / 936
3 / 936
4 / 936
5 / 936
6 / 936
7 / 936
8 / 936
9 / 936
10 / 936
11 / 936
12 / 936
13 / 936
14 / 936
15 / 936
16 / 936
17 / 936
18 / 936
19 / 936
20 / 936
21 / 936
22 / 936
23 / 936
24 / 936
25 / 936
26 / 936
27 / 936
28 / 936
29 / 936
30 / 936
31 / 936
32 / 936
33 / 936
34 / 936
35 / 936
36 / 936
37 / 936
38 / 936
39 / 936
40 / 936
41 / 936
42 / 936
43 / 936
44 / 936
45 / 936
46 / 936
47 / 936
48 / 936
49 / 936
50 / 936
51 / 936
52 / 936
53 / 936
54 / 936
55 / 936
56 / 936
57 / 936
58 / 936
59 / 936
60 / 936
61 / 936
62 / 936
63 / 936
64 / 936
65 / 936
66 / 936
67 / 936
68 / 936
69 / 936
70 / 936
71 / 936
72 / 936
73 / 936
74 / 936
75 / 936
76 / 936
77 / 936
78 / 936
79 / 936
80 / 936
81 / 936
82 / 936
83 / 936
84 / 936
85 / 936
86 / 936
87 / 936
88 / 936
89 / 936
90 / 936
91 / 936
92 / 936
93 / 936
94 / 936
95 / 936
96 / 936
97 / 936
98 / 936
99 / 936
100 / 936
101 / 936
102 / 936
103 / 936
104 / 936
105 / 936
106 / 936
107 / 936
108 / 936
109 / 936
110 / 936
111 / 93

In [ ]:
max_length

49

In [ ]:
for tokenized in tokenizeds:
    padded_input_ids = torch.nn.functional.pad(tokenized['input_ids'], pad=(0, max_length - tokenized['input_ids'].shape[1]))
    padded_attention_mask = torch.nn.functional.pad(tokenized['attention_mask'], pad=(0, max_length - tokenized['attention_mask'].shape[1]))

    tokenized['input_ids'] = padded_input_ids
    tokenized['attention_mask'] = padded_attention_mask

    for k, v in tokenized.items():
        tokenized[k] = v.to(device)

In [ ]:
for i in tokenizeds:
  print(i['attention_mask'].device)
  break

cpu


In [ ]:
folder_path = '/content/drive/My Drive/Data/Targets/'

file_name = 'target'

In [ ]:
import gc

In [ ]:
r = 1
n = len(tokenizeds)
for i in tokenizeds:
  with torch.no_grad():
    x = model(input_ids=i['input_ids'], attention_mask=i['attention_mask']).last_hidden_state[:,0,:]
    x = x.to("cpu")
    (pd.DataFrame(x)).to_csv(folder_path + file_name + str(r) + '.csv', index=False)
    del x
    gc.collect()
    print(r, '/', n)
    r += 1

1 / 936
2 / 936


KeyboardInterrupt: ignored

In [ ]:
folder_path = '/content/drive/My Drive/Data/Targets/'

file_name = 'target1.csv'

df = pd.read_csv(folder_path + file_name)

In [ ]:
folder_path = '/content/drive/My Drive/Data/Scites/'

file_name = 'target'

for i in range(2, 937):
  sdf = pd.read_csv(folder_path+file_name+str(i)+'.csv')
  df = pd.concat([df, sdf], ignore_index=True)
  print(df.shape)

In [ ]:
df.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

normalized_data = scaler.fit_transform(df)

In [ ]:
normalized_data.shape

In [ ]:
(pd.DataFrame(normalized_data)).to_csv('/content/drive/My Drive/Data/scitenorm.csv', index=False)

In [ ]:
ndf = pd.read_csv('/content/drive/My Drive/Data/scitenorm.csv')

In [ ]:
ndf.shape

(93520, 768)

In [ ]:
sdf = pd.read_csv('/content/drive/My Drive/Data/gtdsummarynorm.csv')

In [ ]:
ndf.shape

(93520, 768)